<a href="https://colab.research.google.com/github/kjh0902/recruitment-recommendation-system/blob/main/%EC%B1%84%EC%9A%A9_%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber
!pip install lightfm # 하이브리드 추천 시스템을 위한 대표 라이브러리
!pip install --upgrade gspread google-auth # 구글 시트-코랩 연동을 위한 라이브러리
!pip install --upgrade sentence-transformers

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.font_manager as fm
import warnings
from sentence_transformers import SentenceTransformer
from scipy.sparse import csr_matrix

from lightfm import LightFM
from lightfm.data import Dataset
from scipy.sparse import coo_matrix, csr_matrix

from bs4 import BeautifulSoup
import requests
import re
import io
import pdfplumber
from urllib.parse import urljoin

from google.colab import auth
auth.authenticate_user()

import google.auth
import gspread
from oauth2client.client import GoogleCredentials
from google.oauth2.service_account import Credentials

warnings.simplefilter("ignore")

import logging
logging.getLogger("pdfminer.pdfpage").setLevel(logging.ERROR)

import zipfile

In [ ]:
recruit_df = pd.read_csv("/content/서울시 동대문구 채용 정보.csv", encoding='euc-kr', low_memory=False)

# '제목' 컬럼에 '합격', '면접', '결과' 단어가 들어간 행을 필터링하여 제거
recruit_df = recruit_df[~recruit_df['제목'].str.contains('합격|면접|결과', na=False)]
recruit_df = recruit_df.drop("일련번호", axis=1)
recruit_df = recruit_df.reset_index(drop=True)

recruit_df[:20]

In [ ]:
recruit_df.info()

In [ ]:
link_list = recruit_df['링크'].tolist()
link_list

# 필요한 정보
# 1. user_text: 유저 자신의 취업 정보와 관련된 문장
# 2. item_text: 채용 공고 문장
# 3. interaction_data: 유저가 관심있는 채용 공고 인덱스


In [ ]:
def extract_hwpx_text_from_url(url):
    # HTTP GET
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()

    # 메모리 ZIP 열기
    hwpx_bytes = io.BytesIO(resp.content)
    with zipfile.ZipFile(hwpx_bytes) as z:
        # Preview 텍스트 파일 경로
        preview_path = "Preview/PrvText.txt"
        if preview_path not in z.namelist():
            raise FileNotFoundError(f"{preview_path} 미존재: 내부파일목록={z.namelist()}")
        raw = z.read(preview_path)
        return raw.decode('utf-8')

def extract_pdf_text_from_url(url):
  headers = {"User-Agent": "Mozilla/5.0"}
  response = requests.get(full_url, headers=headers)

  # PDF 파일 다운로드 (메모리 상에서 처리)
  pdf_bytes = io.BytesIO(response.content)

  # pdfplumber로 텍스트 추출
  text = ""
  with pdfplumber.open(pdf_bytes) as pdf:
      for page in pdf.pages:
          page_text = page.extract_text()
          if page_text:
              text += page_text + "\n"
  return text

def detect_type(url):
    resp = requests.get(url, headers={"User-Agent":"Mozilla/5.0"}, stream=True)
    resp.raise_for_status()
    magic = resp.raw.read(4)


    if magic.startswith(b"%PDF"):
        return "pdf"
    if magic.startswith(b"PK\x03\x04"):
        return "hwpx"
    return "unknown"

In [ ]:
# 2. item_text : '링크' 열의 채용 공고 pdf 파일의 텍스트 추출
item_texts = []
total_links = len(link_list)

for idx, i in enumerate(link_list, start=1):
    try:
        base_url = i  # 절대 url

        # headers 추가해서 html 가져오기
        headers = {"User-Agent": "Mozilla/5.0"}
        r = requests.get(base_url, headers=headers)

        # HTML 파싱
        soup = BeautifulSoup(r.text, 'html.parser')

        # downloadBbsFile.do 가 포함된 모든 <a> 태그 중, 내부 텍스트에 "공고"가 포함된 태그 찾기
        link_tag = None
        for tag in soup.find_all('a', href=lambda x: x and 'downloadBbsFile.do' in x):
            if '공고' in tag.get_text():
                link_tag = tag
                break

        # 상대 주소 및 절대 URL 구성
        relative_url = link_tag['href']
        full_url = urljoin(base_url, relative_url)
        file_ext = detect_type(full_url)
        if (file_ext == "hwpx"):
            text = extract_hwpx_text_from_url(full_url)
            item_texts.append(text)
        elif (file_ext == "pdf"):
            text = extract_pdf_text_from_url(full_url)
            item_texts.append(text)
        else:
            raise ValueError()

    except Exception as e:
        # 예외 발생 시, recruit_df에서 제목, 내용, 본문 텍스트 추출
        base_url = i  # 절대 url
        headers = {"User-Agent": "Mozilla/5.0"}
        r = requests.get(base_url, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        content_element = soup.find("td", class_="p-table__content", title="내용")
        main_text = content_element.get_text(separator="\n", strip=True) if content_element else ""
        row = recruit_df[recruit_df['링크'] == i]
        text = str(row['제목'].iloc[0]) + " " + str(row['내용'].iloc[0]) + " " + main_text
        item_texts.append(text)

    # 10개마다 진행 상황 출력
    if idx % 10 == 0:
        print(f"진행 상황: {total_links}개 중 {idx}개 처리됨.")


In [ ]:
# item_texts를 pickle 객체로 저장

import pickle

with open("item_texts.ver2.pkl", "wb") as f:
    pickle.dump(item_texts, f)

In [ ]:
# pickle 객체를 다시 item_texts로 복구

import pickle

# 파일을 읽기 모드로 열어서 pickle 객체를 로드합니다.
with open('/content/item_texts.ver2.pkl', 'rb') as file:
    item_texts = pickle.load(file)

In [ ]:
item_texts

In [ ]:
## 전처리에 필요한 라이브러리 설치

!apt-get update
!apt-get install -y openjdk-8-jdk-headless
!pip install konlpy jpype1

import re
import pickle
from konlpy.tag import Okt
from tqdm import tqdm

In [ ]:
## 전처리 적용

okt = Okt()

# 도메인 불용어 집합
domain_stopwords = {
    # 문서 메타·제목 관련
    '서울특별시', '동대문구', '동대문구청', '동대문구청장', '동대문구인사위원회',
    '제', '호', '공고', '재공고', '추가모집', '모집공고',
    # 채용·근무 포맷
    '채용', '근로자', '기간제', '기간제근로자', '사업', '참여자', '인원', '직무내용',
    '근무기간', '근무시간', '임용', '임용분야', '담당직무', '제출서류',
    # 일반 안내 문구
    '붙임', '첨부', '내용', '제출', '방법', '기타', '관련', '안내', '확인', '문의',
    # 웹·링크
    'http', 'https', 'www', '링크'
 }

def preprocess(text):
    # 줄바꿈, nbsp 정리
    text = text.replace('\xa0', ' ')
    text = re.sub(r'\s+', ' ', text).strip()

    # 토큰 필터링 적용
    tokens = okt.morphs(text)
    filtered = []
    for t in tokens:
        # 숫자, 짧은 거, 도메인 불용어 걸러내기
        if t.isdigit():
            continue
        if len(t) <= 1:
            continue
        if t in domain_stopwords:
            continue
        filtered.append(t)


    return " ".join(filtered)  # 다시 문장으로 합쳐서 반환

processed_texts = [preprocess(t) for t in tqdm(item_texts)]

In [ ]:
item_texts = processed_texts

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sentence_transformers import models

# 임베딩 입력 -> 실루엣 스코어를 반환
# 응집도(cohesion)와 분리도(separation)를 종합해 −1에서 1 사이 점수로 반환(1에 가까울수록 좋음)
def eval_emb(embs):
    km = KMeans(n_clusters=5, random_state=42, n_init=10)
    labels = km.fit_predict(embs)
    return silhouette_score(embs, labels)

configs = ['sentence-transformers/all-MiniLM-L6-v2',
           'jhgan/ko-sroberta-multitask',
           'snunlp/KR-SBERT-V40K-klueNLI-augSTS',
           'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
           'sentence-transformers/LaBSE'
]

results = []
for c in configs:
    print(f"{c} 평가 시작…")
    tm = models.Transformer(c, max_seq_length=128)
    pm = models.Pooling(tm.get_word_embedding_dimension(), pooling_mode='mean')
    m  = SentenceTransformer(modules=[tm, pm])
    embs = m.encode(item_texts, show_progress_bar=False, convert_to_numpy=True)
    score = eval_emb(embs)
    results.append((c, score))


In [ ]:
df = pd.DataFrame(results, columns=['model', 'silhouette'])
print("최종 실험 결과 (실루엣 스코어 기준 내림차순):")
print(df.reset_index(drop=True))

In [ ]:
plt.figure()
plt.bar(df['model'], df['silhouette'])
plt.xticks(rotation=45, ha='right')
plt.ylabel('Silhouette Score')
plt.title('silhouette scores by model')
plt.tight_layout()
plt.show()

In [ ]:
# 2. SBERT로 임베딩 (두 텍스트 모두 동일 모델 사용)
sbert_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 채용 공고 텍스트 임베딩 (결과 shape: (n_items, embedding_dim))
item_embeddings = sbert_model.encode(item_texts, normalize_embeddings=True)

# 3. LightFM이 요구하는 sparse matrix 형태로 변환
item_features = csr_matrix(item_embeddings)

In [ ]:
item_embeddings

In [ ]:
item_features

In [ ]:
# 1. user_text, 3. interaction_data : 구글폼에서 입력받아서 구글 스프레드시트 가져오기

# Google Sheets API를 사용할 범위 지정
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# 기본 인증을 사용하는 경우
gc = gspread.authorize(google.auth.default(scopes=scopes)[0])

# 스프레드시트 열기 (문서 URL이나 제목을 사용)
spreadsheet = gc.open("구직자 응답")  # 스프레드시트 제목 사용
worksheet = spreadsheet.sheet1  # 첫 번째 시트 선택

# 데이터 가져오기
data = worksheet.get_all_records()
job_seekers = pd.DataFrame(data)

# 구직자 프로필 텍스트 생성
def create_profile_text(row):
    profile = f"직무: {job_seekers.loc[row, '직무']} 경력: {job_seekers.loc[row, '경력']} 자격증: {job_seekers.loc[row, '자격증']} 전공: {job_seekers.loc[row, '전공']} 기술: {job_seekers.loc[row, '기술']}"
    return profile

n = 3 # 구글 시트에서 몇번째 사람 데이터인지
user_text = create_profile_text(n)

input_str = job_seekers.loc[n, '관심 있는 공고 인덱스']

# 정규표현식을 사용하여 문자열에서 숫자(연속된 숫자들)를 추출합니다.
numbers = re.findall(r'\d+', input_str)

# 추출된 문자열 숫자들을 정수로 변환하고, ('user_1', 숫자) 형태의 튜플로 리스트를 생성합니다.
interaction_data = [('user_1', int(num)) for num in numbers]

user_text

In [ ]:
# 2. SBERT로 임베딩 (두 텍스트 모두 동일 모델 사용)
sbert_model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# 유저 텍스트 임베딩 (결과 shape: (1, embedding_dim))
user_embedding = sbert_model.encode([user_text], normalize_embeddings=True)

# 3. LightFM이 요구하는 sparse matrix 형태로 변환
user_features = csr_matrix(user_embedding)
item_features = csr_matrix(item_embeddings)

# 4. Dataset 생성
# 유저는 한 명, 아이템은 채용 공고 리스트의 인덱스로 사용
dataset = Dataset()
dataset.fit(users=['user_1'], items=list(range(len(item_texts))))

# 5. interaction 생성
(interactions, _) = dataset.build_interactions(interaction_data)

# 6. LightFM 모델 학습 (user_features와 item_features 모두 사용)
model = LightFM(loss='warp')
model.fit(interactions, user_features=user_features, item_features=item_features,
          epochs=30, num_threads=2)

# 7. 유저에 대해 모든 공고 점수 예측 후 추천 순위 도출
scores = model.predict(0, np.arange(len(item_texts)), user_features=user_features, item_features=item_features)
recommended_order = np.argsort(-scores)

print("<추천 결과>")
for idx in recommended_order[:5]:
    print(f"{idx}: {recruit_df.loc[idx, '제목']} (score: {scores[idx]:.4f})")